In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv
/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip
/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip


# **Prepare Data**

In [2]:
!pip install datasets evaluate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.4.2
    Uninstalling datasets-4.4.2:
      Successfully uninstalled datasets-4.4.2


In [23]:
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoConfig,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

from datasets import (
    Dataset,
    DatasetDict,
    Features, Sequence, ClassLabel, Value
)
import zipfile
import evaluate
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
zipfile.ZipFile("/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip").extractall("test")
zipfile.ZipFile("/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip").extractall("train")

In [5]:
train_df = pd.read_csv("train/train.tsv", sep="\t")
test_df = pd.read_csv("test/test.tsv", sep="\t")
sub_df = pd.read_csv("/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv")

# **Explore Data**

In [6]:
train_df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [7]:
test_df.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [8]:
train_df.shape,test_df.shape

((156060, 4), (66292, 3))

In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PhraseId    156060 non-null  int64 
 1   SentenceId  156060 non-null  int64 
 2   Phrase      156060 non-null  object
 3   Sentiment   156060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.8+ MB


In [10]:
train_df['Sentiment'].nunique()

5

# **Model**

In [11]:
model_name = "microsoft/deberta-v3-large"
config = AutoConfig.from_pretrained(model_name,num_labels=5)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
train_split, dev_split = train_test_split(train_df, test_size=0.2, random_state=42)
print(f'Train set size: {len(train_split)}')
print(f'Dev set size: {len(dev_split)}')

Train set size: 124848
Dev set size: 31212


In [15]:
features = Features({
    "Phrase": Value("string"),
    "Sentiment": ClassLabel(num_classes=5)
})

dataset = DatasetDict({
    'train': Dataset.from_pandas(train_split[['Phrase', 'Sentiment']], features=features, preserve_index=False),
    'dev':   Dataset.from_pandas(dev_split[['Phrase', 'Sentiment']], features=features, preserve_index=False),
})
dataset

DatasetDict({
    train: Dataset({
        features: ['Phrase', 'Sentiment'],
        num_rows: 124848
    })
    dev: Dataset({
        features: ['Phrase', 'Sentiment'],
        num_rows: 31212
    })
})

In [16]:
def tokenize(examples):
    return tokenizer(examples['Phrase'], truncation=True,max_length=128)
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

Map:   0%|          | 0/124848 [00:00<?, ? examples/s]

Map:   0%|          | 0/31212 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Phrase', 'Sentiment', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 124848
    })
    dev: Dataset({
        features: ['Phrase', 'Sentiment', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 31212
    })
})

In [24]:
data_collator = DataCollatorWithPadding(tokenizer,padding=True, pad_to_multiple_of=8)
tokenized_dataset["train"].column_names

['Phrase', 'labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [19]:
tokenized_dataset['dev'][20]

{'Phrase': 'enough to keep us',
 'Sentiment': 2,
 'input_ids': [1, 618, 264, 548, 381, 2],
 'token_type_ids': [0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [20]:
tokenized_dataset = tokenized_dataset.rename_column("Sentiment", "labels")

In [ ]:
args = TrainingArguments(
    output_dir='prachatai-headline-2',
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    fp16=True, 
    num_train_epochs=2,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    eval_strategy='steps',
    eval_steps=500,
    logging_steps=50,
    save_strategy='steps',
    report_to="none"
)

def compute_metrics(eval_preds):
  metric = evaluate.load("accuracy")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["dev"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Step,Training Loss,Validation Loss


# **Submission**

In [ ]:
predictions = trainer.predict(tokenized_dataset['test'])
preds_class = np.argmax(predictions.predictions, axis=-1)
print("Sample predictions:", preds_class[:10])

In [ ]:
submission = pd.DataFrame({
    'PhraseId': test_df['PhraseId'],
    'Sentiment': preds_class
})
print(submission.head())
submission.to_csv('submission.csv', index=False)
submission.head()